In [159]:
# Project Luther

In [160]:
# The Number - get title, budget
# Box office - get title, distributor, mpaa rating, genre, domestic total gross, worldwide gross

In [253]:
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup

In [254]:
url  = 'https://www.boxofficemojo.com/yearly/chart/?page=<page_number>&view=releasedate&view2=domestic&yr=2018&p=.htm'
# Function that gets the urls of all pages for 2018
def generate_urls(url):
    urls = []

    for page_number in range(1,4):
        url_pages = url.replace('<page_number>', str(page_number))
        urls.append(url_pages)
    return urls
urls = generate_urls(url)
urls

['https://www.boxofficemojo.com/yearly/chart/?page=1&view=releasedate&view2=domestic&yr=2016&p=.htm',
 'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2016&p=.htm',
 'https://www.boxofficemojo.com/yearly/chart/?page=3&view=releasedate&view2=domestic&yr=2016&p=.htm']

In [255]:
# Function that returns rows in movie table 
def get_url_text(url_page):
    response    = requests.get(url_page)
    status_code = response.status_code
    page        = response.text
    soup        = BeautifulSoup(page,"lxml")
    tables=soup.find_all("table")
    rows=[row for row in tables[6].find_all('tr')]
    return rows

# Execute function for a sample url
# temp_url = 'https://www.boxofficemojo.com/yearly/chart/?page=2&view=releasedate&view2=domestic&yr=2018&p=.htm'
# text_rows = get_url_text(temp_url)

In [256]:
# Function that returns movie paths
def parse_movie_paths(rows):
    movie_paths = []
    for row in rows[2:102]:
        items      = row.find_all('td')
        movie_path = items[1].find('a')['href']
        movie_paths.append(movie_path)
    return movie_paths

# Fetch all movie paths and assign it to movie_paths var
movie_paths = parse_movie_paths(text_rows)

In [257]:
# Function that gets the response text
def get_movie_page_text(movie_path):
    movie_url           = 'https://www.boxofficemojo.com' + movie_path
    movie_page_response = requests.get(movie_url)
    movie_page_text     = movie_page_response.text
    return movie_page_text

# Execute function for a sample movie_path
movie_page_text = get_movie_page_text('/movies/?id=theincredibles2.htm')

In [258]:
# Scrape movie path to get the production budget from the-numbers website
def fetch_production_budget_movie_path(movie_title):
    url = 'https://www.the-numbers.com/search?searchterm=<movie_title>'
    url_page = url.replace('<movie_title>', movie_title)
    response = requests.get(url_page)
    response.status_code
    page = response.text
    from bs4 import BeautifulSoup
    soup   = BeautifulSoup(page,"lxml")
    tables = soup.find_all("table")[1]
    item   = tables.find_all('td')[2]
    path   = item.find('a')['href']
    return path

In [259]:
# Scrape production budget from the-numbers website
def fetch_production_budget_from_path(path):
    url = 'https://www.the-numbers.com'
    url_page = url + path
    response = requests.get(url_page)
    response.status_code
    page = response.text
    from bs4 import BeautifulSoup
    soup   = BeautifulSoup(page,"lxml")
    tables = soup.find_all("table")[3]
    budget = tables.find_all('td')[1].text
    production_budget = budget.replace('$', '').replace(',', '')
    return int(production_budget)

In [260]:
# Return production budget except return nan
def fetch_production_budget(movie_title):
    try:
        path   = fetch_production_budget_movie_path(movie_title)
        budget = fetch_production_budget_from_path(path)
        return budget     
    except:
        return 'NaN'

In [261]:
# Scrape title, distributor, total gross, opening_weekend, genre and rating for each movie 
def parse_movie_page(movie_page_text):
    soup = BeautifulSoup(movie_page_text, "lxml")
    tables = soup.find_all('table')[5]

    # Get the movie title
    title_string = (soup.find('title')).text
    title        = title_string.split('(')[0].strip()
    
    # Get domestic_total_gross
    gross        = tables.find_all('td')[0].text
    total_gross  = gross.replace(' (Estimate)', '').replace('Domestic Lifetime Gross', '').split(':')[1].strip()
    
    # Get opening weekend
    search_text         = "Opening" + u'\xa0' + "Weekend:"
    opening_weekend_link = soup.body.find('a', text=search_text)
    if opening_weekend_link != None:
        opening_weekend = fetch_amount(opening_weekend_link.parent.find_next_sibling('td').text)
    else:
        opening_weekend = 'NaN'
    
    # Get worldwide gross
    search_ww_text = "Worldwide:"
    ww_gross = soup.body.find('b', text=search_ww_text)
    if ww_gross != None:
        worldwide_gross =fetch_amount(ww_gross.parent.find_next_sibling('td').text)
    else:
        worldwide_gross ='Nan'
    
    # Get distributor
    dist_value   = tables.find_all('td')[1].text
    distributor  = dist_value.split(':')[1].strip()
    
    # Get genre
    genre_value  = tables.find_all('td')[3].text
    genre        = genre_value.split(':')[1].strip()
    
    # Get rating
    rating_value = tables.find_all('td')[5].text
    rating       = rating_value.split(':')[1].strip()
    
    # Get production budget
    budget = tables.find_all('td')[1].text

    production_budget    = fetch_production_budget(title)
    domestic_total_gross = fetch_amount(total_gross)

    # List of headers
    headers = ['movie_title', 'distributor', 'domestic_total_gross','opening_weekend', 'production_budget', 'rating', 'genre', 'worldwide_gross']
    
    # Zip headers and values - create dictionary
    movie_dict = dict(zip(headers, [title, distributor, domestic_total_gross, opening_weekend, production_budget, rating, genre, worldwide_gross]))

    return movie_dict

In [262]:
# Return integer amount of domestic_total_gross
def fetch_amount(amount):
    if amount == None:
        return 'NaN'
    amount = amount.replace('$', '').replace(',', '')
    return int(float(amount))

In [263]:
import time

# Append all the movies to movies list
movies = []
urls   = generate_urls(url)
for url in urls:
    text_rows   = get_url_text(url)
    movie_paths = parse_movie_paths(text_rows)
    #print(movie_paths)
    for movie_path in movie_paths:
        movie_page_text = get_movie_page_text(movie_path)
        movie           = parse_movie_page(movie_page_text)     
        #print(movie)
        movies.append(movie)
    time.sleep(1.5)
       

['/movies/?id=starwars2016.htm', '/movies/?id=pixar2015.htm', '/movies/?id=marvel2016.htm', '/movies/?id=illumination2015.htm', '/movies/?id=junglebook2015.htm', '/movies/?id=deadpool2016.htm', '/movies/?id=disney2016.htm', '/movies/?id=superman2015.htm', '/movies/?id=dc2016.htm', '/movies/?id=illumination2016.htm', '/movies/?id=disney1116.htm', '/movies/?id=fantasticbeasts.htm', '/movies/?id=marvel716.htm', '/movies/?id=hiddenfigures.htm', '/movies/?id=bourne5.htm', '/movies/?id=startrek2016.htm', '/movies/?id=x-men2016.htm', '/movies/?id=trolls.htm', '/movies/?id=lalaland.htm', '/movies/?id=kungfupanda3.htm', '/movies/?id=ghostbusters2016.htm', '/movies/?id=centralintelligence.htm', '/movies/?id=tarzan2016.htm', '/movies/?id=sully.htm', '/movies/?id=untitledlucasmoore.htm', '/movies/?id=angrybirds.htm', '/movies/?id=id42.htm', '/movies/?id=conjuring2.htm', '/movies/?id=arrival2016.htm', '/movies/?id=passengers2016.htm', '/movies/?id=sausageparty.htm', '/movies/?id=themagnificentseven

{'movie_title': 'The Legend of Tarzan', 'distributor': 'Warner Bros.', 'domestic_total_gross': 126643061, 'opening_weekend': 38527856, 'production_budget': 180000000, 'rating': 'PG-13', 'genre': 'Action / Adventure', 'worldwide_gross': 356700357}
{'movie_title': 'Sully', 'distributor': 'Warner Bros.', 'domestic_total_gross': 125070033, 'opening_weekend': 35028301, 'production_budget': 60000000, 'rating': 'PG-13', 'genre': 'Drama', 'worldwide_gross': 240797623}
{'movie_title': 'Bad Moms', 'distributor': 'STX Entertainment', 'domestic_total_gross': 113257297, 'opening_weekend': 23817340, 'production_budget': 20000000, 'rating': 'R', 'genre': 'Comedy', 'worldwide_gross': 183936074}
{'movie_title': 'The Angry Birds Movie', 'distributor': 'Sony / Columbia', 'domestic_total_gross': 107509366, 'opening_weekend': 38155177, 'production_budget': 73000000, 'rating': 'PG', 'genre': 'Animation', 'worldwide_gross': 352333929}
{'movie_title': 'Independence Day: Resurgence', 'distributor': 'Fox', 'dom

{'movie_title': 'The BFG', 'distributor': 'Buena Vista', 'domestic_total_gross': 55483770, 'opening_weekend': 18775350, 'production_budget': 140000000, 'rating': 'PG', 'genre': 'Adventure', 'worldwide_gross': 183345589}
{'movie_title': 'Neighbors 2: Sorority Rising', 'distributor': 'Universal', 'domestic_total_gross': 55455765, 'opening_weekend': 21760405, 'production_budget': 35000000, 'rating': 'R', 'genre': 'Comedy', 'worldwide_gross': 108007109}
{'movie_title': 'The Shallows', 'distributor': 'Sony / Columbia', 'domestic_total_gross': 55124043, 'opening_weekend': 16800868, 'production_budget': 13000000, 'rating': 'PG-13', 'genre': 'Horror', 'worldwide_gross': 119100758}
{'movie_title': 'Office Christmas Party', 'distributor': 'Paramount', 'domestic_total_gross': 54767494, 'opening_weekend': 16890204, 'production_budget': 45000000, 'rating': 'R', 'genre': 'Comedy', 'worldwide_gross': 114501299}
{'movie_title': "Assassin's Creed", 'distributor': 'Fox', 'domestic_total_gross': 54647948

{'movie_title': 'Hell or High Water', 'distributor': 'Lionsgate', 'domestic_total_gross': 27007844, 'opening_weekend': 'NaN', 'production_budget': 12000000, 'rating': 'R', 'genre': 'Action Drama', 'worldwide_gross': 37879877}
{'movie_title': 'The Forest', 'distributor': 'Focus Features', 'domestic_total_gross': 26594261, 'opening_weekend': 12741176, 'production_budget': 10000000, 'rating': 'PG-13', 'genre': 'Horror', 'worldwide_gross': 37608299}
{'movie_title': 'Ben-Hur', 'distributor': 'Paramount', 'domestic_total_gross': 26410477, 'opening_weekend': 11203815, 'production_budget': 15000000, 'rating': 'PG-13', 'genre': 'Historical Epic', 'worldwide_gross': 94061311}
{'movie_title': 'The Witch', 'distributor': 'A24', 'domestic_total_gross': 25138705, 'opening_weekend': 8800230, 'production_budget': 3500000, 'rating': 'R', 'genre': 'Horror', 'worldwide_gross': 40423945}
{'movie_title': "Bridget Jones's Baby", 'distributor': 'Universal', 'domestic_total_gross': 24252420, 'opening_weekend'

{'movie_title': 'The Infiltrator', 'distributor': 'Broad Green Pictures', 'domestic_total_gross': 15436808, 'opening_weekend': 5303775, 'production_budget': 47500000, 'rating': 'R', 'genre': 'Drama / Thriller', 'worldwide_gross': 15436808}
{'movie_title': 'A Beautiful Planet', 'distributor': 'IMAX', 'domestic_total_gross': 15141934, 'opening_weekend': 223983, 'production_budget': 'NaN', 'rating': 'G', 'genre': 'Documentary', 'worldwide_gross': 23408402}
{'movie_title': 'Keeping Up with the Joneses', 'distributor': 'Fox', 'domestic_total_gross': 14904426, 'opening_weekend': 5461475, 'production_budget': 'NaN', 'rating': 'PG-13', 'genre': 'Comedy', 'worldwide_gross': 29918745}
{'movie_title': 'Criminal', 'distributor': 'Lionsgate/Summit', 'domestic_total_gross': 14708696, 'opening_weekend': 5767278, 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Action', 'worldwide_gross': 38803993}
{'movie_title': 'Hello, My Name is Doris', 'distributor': 'Roadside Attractions', 'domestic_total_gr

{'movie_title': 'Bleed for This', 'distributor': 'Open Road Films', 'domestic_total_gross': 5083906, 'opening_weekend': 2366810, 'production_budget': 16000000, 'rating': 'R', 'genre': 'Sports Drama', 'worldwide_gross': 5083906}
{'movie_title': 'Incarnate', 'distributor': 'BH Tilt', 'domestic_total_gross': 4799774, 'opening_weekend': 2534884, 'production_budget': 5000000, 'rating': 'PG-13', 'genre': 'Horror', 'worldwide_gross': 6341855}
{'movie_title': 'Absolutely Fabulous: The Movie', 'distributor': 'Fox Searchlight', 'domestic_total_gross': 4772155, 'opening_weekend': 1861118, 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Comedy', 'worldwide_gross': 37915971}
{'movie_title': 'Hands of Stone', 'distributor': 'Weinstein Company', 'domestic_total_gross': 4712792, 'opening_weekend': 1751388, 'production_budget': 20000000, 'rating': 'R', 'genre': 'Drama', 'worldwide_gross': 4712792}
{'movie_title': "Don't Think Twice", 'distributor': 'The Film Arcade', 'domestic_total_gross': 441766

{'movie_title': 'The Disappointments Room', 'distributor': 'Relativity', 'domestic_total_gross': 2423468, 'opening_weekend': 1402823, 'production_budget': 15000000, 'rating': 'R', 'genre': 'Horror', 'worldwide_gross': 2423468}
{'movie_title': 'Hillsong - Let Hope Rise', 'distributor': 'Pure Flix', 'domestic_total_gross': 2394386, 'opening_weekend': 1357243, 'production_budget': 'NaN', 'rating': 'PG', 'genre': 'Documentary', 'worldwide_gross': 2649761}
{'movie_title': 'Dear Zindagi', 'distributor': 'Reliance Big Pictures', 'domestic_total_gross': 2376479, 'opening_weekend': 977862, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Comedy', 'worldwide_gross': 2376479}
{'movie_title': 'Elle', 'distributor': 'Sony Classics', 'domestic_total_gross': 2341534, 'opening_weekend': 50934, 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Drama / Thriller', 'worldwide_gross': 2341534}
{'movie_title': 'A Aa', 'distributor': 'Blue Sky Cinema', 'domestic_total_gross': 2313043, 'opening_w

{'movie_title': 'The Comedian', 'distributor': 'Sony Classics', 'domestic_total_gross': 1659002, 'opening_weekend': 'NaN', 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Comedy', 'worldwide_gross': 'Nan'}
{'movie_title': 'I Saw the Light', 'distributor': 'Sony Classics', 'domestic_total_gross': 1646788, 'opening_weekend': 'NaN', 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Drama', 'worldwide_gross': 1646788}
{'movie_title': 'Equity', 'distributor': 'Sony Classics', 'domestic_total_gross': 1605463, 'opening_weekend': 82434, 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Drama', 'worldwide_gross': 'Nan'}
{'movie_title': 'Neerja', 'distributor': 'FIP', 'domestic_total_gross': 1585687, 'opening_weekend': 626528, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Drama', 'worldwide_gross': 17618280}
{'movie_title': 'Jane Got a Gun', 'distributor': 'Weinstein Company', 'domestic_total_gross': 1513793, 'opening_weekend': 835572, 'production_budget': 25000000, 'rati

{'movie_title': 'Chongqing Hot Pot', 'distributor': 'China Lion', 'domestic_total_gross': 779818, 'opening_weekend': 201960, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Thriller', 'worldwide_gross': 779818}
{'movie_title': 'The Polar Express', 'distributor': 'Warner Bros.', 'domestic_total_gross': 765154, 'opening_weekend': 'NaN', 'production_budget': 170000000, 'rating': 'G', 'genre': 'Animation', 'worldwide_gross': 186493472}
{'movie_title': 'I Belonged to You', 'distributor': 'China Lion', 'domestic_total_gross': 744541, 'opening_weekend': 311738, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Romance', 'worldwide_gross': 'Nan'}
{'movie_title': "Saturday's Warrior", 'distributor': 'Purdie Distribution', 'domestic_total_gross': 744046, 'opening_weekend': 96041, 'production_budget': 'NaN', 'rating': 'PG', 'genre': 'Musical', 'worldwide_gross': 'Nan'}
{'movie_title': 'The Monkey King 2 in 3D', 'distributor': 'China Lion', 'domestic_total_gross': 709982, 'open

{'movie_title': 'Only Yesterday', 'distributor': 'GKIDS', 'domestic_total_gross': 453243, 'opening_weekend': 14970, 'production_budget': 'NaN', 'rating': 'PG', 'genre': 'Animation', 'worldwide_gross': 453243}
{'movie_title': 'Gimme Danger', 'distributor': 'Magnolia', 'domestic_total_gross': 440627, 'opening_weekend': 44725, 'production_budget': 'NaN', 'rating': 'R', 'genre': 'Documentary', 'worldwide_gross': 'Nan'}
{'movie_title': 'Baaghi', 'distributor': 'UTV Communications', 'domestic_total_gross': 438207, 'opening_weekend': 237215, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Action Comedy', 'worldwide_gross': 438207}
{'movie_title': 'I Am Not Madame Bovary', 'distributor': 'Well Go USA', 'domestic_total_gross': 436798, 'opening_weekend': 181552, 'production_budget': 'NaN', 'rating': 'Unrated', 'genre': 'Comedy / Drama', 'worldwide_gross': 70826207}
{'movie_title': 'Land of Mine', 'distributor': 'Sony Classics', 'domestic_total_gross': 435266, 'opening_weekend': 'NaN',

In [287]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [265]:
df = pd.DataFrame(movies)

In [266]:
df.head()

,distributor,domestic_total_gross,genre,movie_title,opening_weekend,production_budget,rating,worldwide_gross
0,Buena Vista,532177324,Sci-Fi Adventure,Rogue One: A Star Wars Story,155081681,200000000,PG-13,1056057273
1,Buena Vista,486295561,Animation,Finding Dory,135060273,200000000,PG,1028570889
2,Buena Vista,408084349,Action / Adventure,Captain America: Civil War,179139142,250000000,PG-13,1153304495
3,Universal,368384330,Animation,The Secret Life of Pets,104352905,75000000,PG,875457937
4,Buena Vista,364001123,Adventure,The Jungle Book,103261464,NaN,PG,966550600


In [267]:
df.to_pickle('/Users/gozdeturan/Desktop/sf18_ds12/projects/02-luther/movies2016.pickle')

In [268]:
# df.to_csv('movies2016.csv', sep=',', encoding='utf-8')